In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

C:\Users\arjyo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:

import pandas as pd


# Replace with the filepath to your csv file
file_path = "C:/Users/arjyo/simulation/unnamed/openstudio/run/epluszsz.csv"

# Read the Excel file into a DataFrame
df = pd.read_csv(file_path)


df.head()

,Time,ROOM_1_A16AB98E:CANBERRA ANN HTG 99.6% CONDNS DB:Des Heat Load [W],ROOM_1_A16AB98E:CANBERRA ANN CLG .4% CONDNS DB=>MWB:Des Sens Cool Load [W],ROOM_1_A16AB98E:CANBERRA ANN HTG 99.6% CONDNS DB:Des Heat Mass Flow [kg/s],ROOM_1_A16AB98E:CANBERRA ANN CLG .4% CONDNS DB=>MWB:Des Cool Mass Flow [kg/s],ROOM_1_A16AB98E::Des Latent Heat Load [W],ROOM_1_A16AB98E::Des Latent Cool Load [W],ROOM_1_A16AB98E::Des Latent Heat Mass Flow [kg/s],ROOM_1_A16AB98E::Des Latent Cool Mass Flow [kg/s],ROOM_1_A16AB98E::Des Heat Load No DOAS [W],ROOM_1_A16AB98E::Des Sens Cool Load No DOAS [W],ROOM_1_A16AB98E::Des Latent Heat Load No DOAS [W],ROOM_1_A16AB98E::Des Latent Cool Load No DOAS [W],ROOM_1_A16AB98E:CANBERRA ANN HTG 99.6% CONDNS DB:Heating Zone Temperature [C],ROOM_1_A16AB98E:CANBERRA ANN HTG 99.6% CONDNS DB:Heating Zone Relative Humidity [%],ROOM_1_A16AB98E:CANBERRA ANN CLG .4% CONDNS DB=>MWB:Cooling Zone Temperature [C],ROOM_1_A16AB98E:CANBERRA ANN CLG .4% CONDNS DB=>MWB:Cooling Zone Relative Humidity [%]
0,00:10:00,0.0,58773.36,0.0,2.742131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,22.65795,35.0,22.65795
1,00:20:00,0.0,58726.31,0.0,2.739936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,22.65795,35.0,22.65795
2,00:30:00,0.0,58680.09,0.0,2.737779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,22.65795,35.0,22.65795
3,00:40:00,0.0,58634.68,0.0,2.735660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,22.65795,35.0,22.65795
4,00:50:00,0.0,58590.05,0.0,2.733578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,22.65795,35.0,22.65795


In [9]:
# Identify columns with all observations as 0
columns_to_remove = df.columns[df.all() == 0]



In [10]:
# Drop columns with all observations as 0
df = df.drop(columns=columns_to_remove)

In [11]:
df.head()

,Time,ROOM_1_A16AB98E:CANBERRA ANN CLG .4% CONDNS DB=>MWB:Des Sens Cool Load [W],ROOM_1_A16AB98E:CANBERRA ANN CLG .4% CONDNS DB=>MWB:Des Cool Mass Flow [kg/s],ROOM_1_A16AB98E:CANBERRA ANN HTG 99.6% CONDNS DB:Heating Zone Temperature [C],ROOM_1_A16AB98E:CANBERRA ANN HTG 99.6% CONDNS DB:Heating Zone Relative Humidity [%],ROOM_1_A16AB98E:CANBERRA ANN CLG .4% CONDNS DB=>MWB:Cooling Zone Temperature [C],ROOM_1_A16AB98E:CANBERRA ANN CLG .4% CONDNS DB=>MWB:Cooling Zone Relative Humidity [%]
0,00:10:00,58773.36,2.742131,35.0,22.65795,35.0,22.65795
1,00:20:00,58726.31,2.739936,35.0,22.65795,35.0,22.65795
2,00:30:00,58680.09,2.737779,35.0,22.65795,35.0,22.65795
3,00:40:00,58634.68,2.735660,35.0,22.65795,35.0,22.65795
4,00:50:00,58590.05,2.733578,35.0,22.65795,35.0,22.65795


In [13]:
df.drop(df.columns[[2, 3,4,5,6]], axis=1, inplace=True)

In [14]:
df.head()

,Time,ROOM_1_A16AB98E:CANBERRA ANN CLG .4% CONDNS DB=>MWB:Des Sens Cool Load [W]
0,00:10:00,58773.36
1,00:20:00,58726.31
2,00:30:00,58680.09
3,00:40:00,58634.68
4,00:50:00,58590.05


In [15]:
new_column_name = 'cool_load'

# Rename the first column
df.rename(columns={df.columns[1]: new_column_name}, inplace=True)

In [16]:
df.head()

,Time,cool_load
0,00:10:00,58773.36
1,00:20:00,58726.31
2,00:30:00,58680.09
3,00:40:00,58634.68
4,00:50:00,58590.05


In [17]:
timestamps = df['Time']
cooling_load_values = df['cool_load']

In [19]:

from sklearn.preprocessing import MinMaxScaler

# Normalize the data using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
cooling_load_values_scaled = scaler.fit_transform(cooling_load_values.values.reshape(-1, 1))


In [20]:
# Define a function to create the LSTM dataset
def create_lstm_dataset(data, look_back=1):
    X, y = [], []
    for i in range(len(data)-look_back):
        X.append(data[i:(i+look_back), 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)


In [21]:
# Convert the time series data to supervised learning format
look_back = 10  # Adjust the look-back window based on your data
X, y = create_lstm_dataset(cooling_load_values_scaled, look_back)

# Reshape the data for LSTM input (samples, time steps, features)
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))


In [29]:
X

array([[[0.16678001, 0.14919964, 0.13192939, ..., 0.0716928 ,
         0.0669063 , 0.06233653]],

       [[0.14919964, 0.13192939, 0.11496181, ..., 0.0669063 ,
         0.06233653, 0.05797226]],

       [[0.13192939, 0.11496181, 0.09828568, ..., 0.06233653,
         0.05797226, 0.05379856]],

       ...,

       [[0.27613329, 0.26460983, 0.253744  , ..., 0.20762028,
         0.19966521, 0.19203521]],

       [[0.26460983, 0.253744  , 0.24349844, ..., 0.19966521,
         0.19203521, 0.18469293]],

       [[0.253744  , 0.24349844, 0.23383577, ..., 0.19203521,
         0.18469293, 1.        ]]])

In [30]:
y

array([0.05797226, 0.05379856, 0.04980047, 0.04598174, 0.04232741,
       0.03884123, 0.03550824, 0.03233219, 0.02927571, 0.02633133,
       0.02348409, 0.02072653, 0.01804744, 0.01543934, 0.01292839,
       0.01051086, 0.00818674, 0.00596724, 0.00384489, 0.00182343,
       0.        , 0.00437174, 0.01845098, 0.04252171, 0.07374789,
       0.10961484, 0.14763029, 0.17954773, 0.20163809, 0.21464869,
       0.22158369, 0.22471117, 0.22629172, 0.22830944, 0.23168727,
       0.23675027, 0.24387583, 0.25353849, 0.26594751, 0.28141301,
       0.29921757, 0.31798616, 0.33717324, 0.35647242, 0.37569686,
       0.39476437, 0.41358154, 0.43205121, 0.45006128, 0.46759308,
       0.48457934, 0.50095655, 0.51668361, 0.53173808, 0.54609757,
       0.55972843, 0.57243637, 0.58407192, 0.59463509, 0.60403246,
       0.61223415, 0.61921025, 0.62527837, 0.63092053, 0.63665237,
       0.64297458, 0.65038785, 0.65947509, 0.67046049, 0.68326931,
       0.69761012, 0.71326244, 0.72989747, 0.74698836, 0.76419

In [33]:
nan_indices = np.isnan(y)
X = X[~nan_indices]
y = y[~nan_indices]

In [46]:
X=X[:-1]


In [47]:
X

array([[[0.16678001, 0.14919964, 0.13192939, ..., 0.0716928 ,
         0.0669063 , 0.06233653]],

       [[0.14919964, 0.13192939, 0.11496181, ..., 0.0669063 ,
         0.06233653, 0.05797226]],

       [[0.13192939, 0.11496181, 0.09828568, ..., 0.06233653,
         0.05797226, 0.05379856]],

       ...,

       [[0.30132124, 0.28835548, 0.27613329, ..., 0.22466259,
         0.21593779, 0.20762028]],

       [[0.28835548, 0.27613329, 0.26460983, ..., 0.21593779,
         0.20762028, 0.19966521]],

       [[0.27613329, 0.26460983, 0.253744  , ..., 0.20762028,
         0.19966521, 0.19203521]]])

In [50]:
y=y[:-1]

In [51]:
y


array([0.05797226, 0.05379856, 0.04980047, 0.04598174, 0.04232741,
       0.03884123, 0.03550824, 0.03233219, 0.02927571, 0.02633133,
       0.02348409, 0.02072653, 0.01804744, 0.01543934, 0.01292839,
       0.01051086, 0.00818674, 0.00596724, 0.00384489, 0.00182343,
       0.        , 0.00437174, 0.01845098, 0.04252171, 0.07374789,
       0.10961484, 0.14763029, 0.17954773, 0.20163809, 0.21464869,
       0.22158369, 0.22471117, 0.22629172, 0.22830944, 0.23168727,
       0.23675027, 0.24387583, 0.25353849, 0.26594751, 0.28141301,
       0.29921757, 0.31798616, 0.33717324, 0.35647242, 0.37569686,
       0.39476437, 0.41358154, 0.43205121, 0.45006128, 0.46759308,
       0.48457934, 0.50095655, 0.51668361, 0.53173808, 0.54609757,
       0.55972843, 0.57243637, 0.58407192, 0.59463509, 0.60403246,
       0.61223415, 0.61921025, 0.62527837, 0.63092053, 0.63665237,
       0.64297458, 0.65038785, 0.65947509, 0.67046049, 0.68326931,
       0.69761012, 0.71326244, 0.72989747, 0.74698836, 0.76419

In [52]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')


In [53]:
# Train the model
model.fit(X, y, epochs=100, batch_size=1, verbose=2)

# Make predictions on the entire dataset
predictions = model.predict(X)

Epoch 1/100
134/134 - 1s - loss: 0.0365 - 887ms/epoch - 7ms/step
Epoch 2/100
134/134 - 0s - loss: 0.0080 - 139ms/epoch - 1ms/step
Epoch 3/100
134/134 - 0s - loss: 0.0065 - 136ms/epoch - 1ms/step
Epoch 4/100
134/134 - 0s - loss: 0.0049 - 139ms/epoch - 1ms/step
Epoch 5/100
134/134 - 0s - loss: 0.0040 - 138ms/epoch - 1ms/step
Epoch 6/100
134/134 - 0s - loss: 0.0023 - 134ms/epoch - 1ms/step
Epoch 7/100
134/134 - 0s - loss: 0.0022 - 137ms/epoch - 1ms/step
Epoch 8/100
134/134 - 0s - loss: 9.0821e-04 - 124ms/epoch - 922us/step
Epoch 9/100
134/134 - 0s - loss: 5.2611e-04 - 150ms/epoch - 1ms/step
Epoch 10/100
134/134 - 0s - loss: 4.0435e-04 - 137ms/epoch - 1ms/step
Epoch 11/100
134/134 - 0s - loss: 3.8684e-04 - 137ms/epoch - 1ms/step
Epoch 12/100
134/134 - 0s - loss: 3.4685e-04 - 136ms/epoch - 1ms/step
Epoch 13/100
134/134 - 0s - loss: 3.2273e-04 - 141ms/epoch - 1ms/step
Epoch 14/100
134/134 - 0s - loss: 3.1965e-04 - 120ms/epoch - 898us/step
Epoch 15/100
134/134 - 0s - loss: 3.3742e-04 - 137ms/

In [54]:
predictions

array([[0.05987694],
       [0.05756286],
       [0.05403166],
       [0.04970814],
       [0.04539358],
       [0.04132394],
       [0.03809944],
       [0.03502317],
       [0.03209611],
       [0.02929068],
       [0.02658898],
       [0.02397197],
       [0.02143022],
       [0.01895517],
       [0.01654128],
       [0.01421039],
       [0.01197181],
       [0.00983164],
       [0.00779628],
       [0.00586299],
       [0.00402989],
       [0.00236814],
       [0.00549652],
       [0.01838697],
       [0.04355089],
       [0.07918674],
       [0.12166031],
       [0.16661169],
       [0.20527005],
       [0.23084004],
       [0.24176179],
       [0.24193878],
       [0.23646939],
       [0.23019724],
       [0.22668897],
       [0.22754957],
       [0.23271759],
       [0.24132058],
       [0.25293475],
       [0.26745823],
       [0.28512204],
       [0.3054504 ],
       [0.32712466],
       [0.3490374 ],
       [0.3704659 ],
       [0.39111802],
       [0.41102546],
       [0.430

In [58]:
from sklearn.metrics import mean_squared_error
import numpy as np

mse = mean_squared_error(y, predictions)
rmse = np.sqrt(mse)
rmse

0.008229656779672372